In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## Scraping

In [ ]:
def scrape_page(page_n):
    url = f"https://www.quandoo.fi/en/helsinki?districtFilter=3637&bookable=true&onlySpecialOffers=false&page={page_n}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rest_cards = soup.find_all(attrs={"data-qa": "merchant-card"})

    rest_names = [card.find('h3').text.strip() for card in rest_cards]
    rest_locations = [card.find(attrs={"data-qa": "merchant-location"}).text.strip() for card in rest_cards]

    # "Located in ... area" part removed in Location column.
    rest_locations = [location.replace('Located at ','').replace('area','') for location in rest_locations]
    rest_cuisines = [card.find(attrs={"data-qa": "merchant-card-cuisine"}).text.strip() for card in rest_cards]
    
    rest_meals_boxes = [card.find(attrs={"data-qa": "merchant-meal"}) for card in rest_cards]
    rest_meals = [box.text.strip() if box else None for box in rest_meals_boxes]
    
    rest_rating_boxes = [card.find(attrs={"data-qa": "reviews-score"}) for card in rest_cards]
    rest_ratings = [float(box.text.split('/')[0].strip()) if box else None for box in rest_rating_boxes]

    rest_review_boxes = [card.find(class_="sc-1atis9w-3 dfyExP") for card in rest_cards]
    rest_review_counts = [int(box.text.split()[0].strip()) if box else None for box in rest_review_boxes]

    rest_price_boxes = [card.find(class_=re.compile(r'.*price-indicator')) for card in rest_cards]
    rest_price_levels = [len(box.find_all(class_=re.compile(r'.*oGCHK'))) for box in rest_price_boxes]

    rest_page_urls = [card.find('a')['href'] for card in rest_cards]

    df = pd.DataFrame({'Name': rest_names, 
                    'Location': rest_locations,
                    'Cuisine': rest_cuisines,
                    'Meals': rest_meals,
                    'Price Level (out of 4)': rest_price_levels,
                    'Rating (out of 6)': rest_ratings,
                    'Review Count': rest_review_counts,
                    'Page URL': rest_page_urls
                    })

    return df

In [20]:
restaurant_data = pd.DataFrame()

for page_n in range(1, 15):
    page_data = scrape_page(page_n)
    restaurant_data = pd.concat([restaurant_data, page_data], ignore_index=True)

restaurant_data

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
0,Luovuus kukkii kaaoksesta,Kaartinkaupunki,International Restaurant,Meals: Dinner,3,5.8,470.0,/en/place/luovuus-kukkii-kaaoksesta-90397/menu
1,Ravintola MyStuu,Punavuori,Swiss Restaurant,"Meals: Lunch, Dessert, Dinner",3,5.7,293.0,/en/place/ravintola-my-stuu-98898/menu
2,Gaucho,City Centre,Brazilian Restaurant,Meals: Dinner,3,5.6,1118.0,/en/place/gaucho-105125/menu
3,Finlandia Caviar,City Centre,Gourmet Restaurant,"Meals: Lunch, Dinner",3,5.6,143.0,/en/place/finlandia-caviar-15896/menu
4,Restaurant Armenian House,Kamppi,International Restaurant,"Meals: Lunch, Dinner",2,5.6,316.0,/en/place/armenian-house-55148/menu
...,...,...,...,...,...,...,...,...
337,Black Sea Kitchen,Kaartinkaupunki,Georgian Restaurant,"Meals: Lunch, Dinner",3,NaN,NaN,/en/place/black-sea-kitchen-109077/about
338,Mad Finn Brewing Co. Taproom Helsinki,Sompasaari,Pizza,Meals: Dinner,2,6.0,2.0,/en/place/mad-finn-brewing-co-taproom-helsinki...
339,Kahvila Mutteri,Lauttasaari,Dessert,Meals: Cake & Coffee,2,NaN,NaN,/en/place/kahvila-mutteri-100773/menu
340,Merisali - Hilton Kalastajatorppa,Munkkiniemi,Scandinavian Restaurant,"Meals: Buffet, Dinner",3,5.0,3.0,/en/place/merisali-hilton-kalastajatorppa-9200...


In [21]:
restaurant_data.to_csv("restaurant_data.csv", sep="\t")

## Prettifying

In [22]:
data = pd.read_csv("restaurant_data.csv", sep="\t", index_col=0)
data.sample(10)

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
192,The Last Drop Bar & Kitchen,Kaartinkaupunki,Eat & Drink,Meals: Dinner,2,5.3,9.0,/en/place/the-last-drop-bar-kitchen-104517/menu
252,Spasso Pizzeria,Alppiharju,Pizza,"Meals: Lunch, Dessert, Dinner",3,NaN,NaN,/en/place/spasso-pizzeria-23968/menu
109,The Tower - Wine & Craft Beer,Pasila,Scandinavian Restaurant,"Meals: Lunch, Dinner",3,4.7,101.0,/en/place/the-tower-wine-craft-beer-85892/menu
260,Origin Ravintola,Sompasaari,Asian Restaurant,"Meals: Lunch, Dinner",2,6.0,1.0,/en/place/origin-ravintola-107918/about
100,Meritorppa,Munkkiniemi,Scandinavian Restaurant,"Meals: Lunch, Dessert, Brunch, Dinner",3,5.0,409.0,/en/place/meritorppa-hilton-kalastajatorppa-63...
233,Hei Noodle,City Centre,Asian Restaurant,"Meals: Lunch, Dinner",3,3.5,NaN,/en/place/hei-noodle-104141/menu
31,Noodle Story Freda,Kamppi,Asian Restaurant,"Meals: Lunch, Dinner",2,5.4,66.0,/en/place/noodle-story-freda-94139/menu
201,Mama's Empanadas & Wine,Kallio,Eat & Drink,"Meals: Breakfast, Tapas, Lunch, Dinner",2,5.0,1.0,/en/place/mama-s-empanadas-wine-104632/menu
340,Merisali - Hilton Kalastajatorppa,Munkkiniemi,Scandinavian Restaurant,"Meals: Buffet, Dinner",3,5.0,3.0,/en/place/merisali-hilton-kalastajatorppa-9200...
262,Presto Pizza Helsinki,Pasila,Pizza,"Meals: Lunch, Dinner",2,5.9,7.0,/en/place/presto-pizza-helsinki-100503/menu
